In [2]:
#import pymongo as pm
import pandas as pd
import numpy as np
#import getpass
import os
import datetime
#from bson.objectid import ObjectId
import pprint
#from inspect import signature
import math
##import scipy.stats as stats
#from scipy.stats import normaltest
#from scipy.stats import kstest
#from distfit import distfit
#import array

#import seaborn as sns
import matplotlib.pyplot as plt
#import networkx as nx
#from cdlib import algorithms
#import random as rd
import re

In [20]:
import requests
payload = {
    "css_object_id": '002075129500006205772020', 
    "n_results": 10,
    "relevant_field": "description_problem_description", 
    "maximum_records": 10000, 
    "sbert_coeff": 200
}
fusion = requests.post('http://198.18.182.160/decision_fusion', json=payload)
#fusion = requests.post('http://100.100.9.8/decision_fusion', json=payload)

fusion_json = fusion.json()

ConnectionError: HTTPConnectionPool(host='198.18.182.160', port=80): Max retries exceeded with url: /decision_fusion (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fdf69bf9c40>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [ ]:
fusion_json

In [12]:
from bs4 import BeautifulSoup
import json
def extract_customer_info(css_id):
    css_object_id = css_id
    print("Starting for " + css_object_id)
    r = requests.post('http://198.18.129.86/query_endpoint', data = {
        'query_id': css_object_id,
        'n_results': 10,
        'text_field': 'description_problem_description',
        'max_records': 10000,
        'sbert_coeff': 200,
        'backtrack_days': 60,
        'strong_threshold': 5,
        'weak_threshold': 3,
        'exclude_spam_solutions':False,
        'filter_by_prod_version':True,
        'filter_by_component':True
        })
    soup = BeautifulSoup(r.text)
    try:
        json_response = json.loads(soup.find(id="json_text").text)
        return json.dumps(json_response)
    except AttributeError:
        return ""

In [9]:
extract_customer_info('002075129500006205772020')

Starting for 002075129500006205772020


ConnectionError: HTTPConnectionPool(host='198.18.129.86', port=80): Max retries exceeded with url: /query_endpoint (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fdf69932c10>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [ ]:
!pip3 install pymongo


In [17]:
import pymongo
import os
from pymongo import MongoClient
import pandas as pd
db_config= {"MONGODB_ADDRESS"    : os.environ.get('KEY_MONGO_ADDRESS','mongodb-service.stag-data.svc.cluster.local'),
            "MONGODB_PORT"       : os.environ.get('KEY_MONGO_PORT', '27017'),
            "MONGODB_USERNAME"   : os.environ.get('KEY_MONGO_USER', 'IBSO-read-write'),
            "MONGODB_PASSWORD"   : os.environ.get('KEY_MONGO_PASSWORD', 'NHWD5bw0xcMgy1Y'),
            "MONGODB_DATABASE"   : os.environ.get('KEY_MONGO_DATABASE', 'admin')}
mongo_client = MongoClient(                ('mongodb://%s:%s@' + db_config["MONGODB_ADDRESS"] + ':' + db_config["MONGODB_PORT"] + '/' + 
                                            db_config["MONGODB_DATABASE"]) % (                    db_config["MONGODB_USERNAME"], 
                                                                              db_config["MONGODB_PASSWORD"]), replicaset='rs0')
solutions_collection = mongo_client["main"]["solutions_parsed_version"]



SyntaxError: invalid character in identifier (<ipython-input-17-113d92cce0e0>, line 3)

In [16]:
from pymongo import MongoClient
import os as os
import pandas as pd
db_config= {
"MONGODB_ADDRESS"    : os.environ.get('KEY_MONGO_ADDRESS','mongodb-service.stag-data.svc.cluster.local'),
"MONGODB_PORT"       : os.environ.get('KEY_MONGO_PORT', '27017'),
"MONGODB_USERNAME"   : os.environ.get('KEY_MONGO_USER', 'IBSO-read-write'),
"MONGODB_PASSWORD"   : os.environ.get('KEY_MONGO_PASSWORD', 'NHWD5bw0xcMgy1Y'),
"MONGODB_DATABASE"   : os.environ.get('KEY_MONGO_DATABASE', 'admin')
}
mongo_client = MongoClient(
                ('mongodb://%s:%s@' + db_config["MONGODB_ADDRESS"] + ':' + db_config["MONGODB_PORT"] + '/' + db_config["MONGODB_DATABASE"]) % (
                    db_config["MONGODB_USERNAME"], db_config["MONGODB_PASSWORD"]), replicaset='rs0')
solutions_collection = mongo_client["main"]["solutions_parsed_version"]
solution_number = '2541754'
all_versions_list = list(solutions_collection.find({'Number': solution_number}).sort('ChangedOn_dt', -1))
all_versions_df = pd.DataFrame(all_versions_list)
latest_version = all_versions_list[0]



ModuleNotFoundError: No module named 'pymongo'

In [ ]:
latest_version

In [ ]:
import pandas as pd
df_all=pd.DataFrame(list(solutions_collection.find({}).limit(1)))

In [ ]:
df_all.columns

In [ ]:
solution_data.shape
solution_data.head(10)

In [ ]:
n = len(pd.unique(solution_data['Version']))
  
print("No.of.unique values :", n)

In [ ]:
pd.unique(solution_data['Category'])

In [ ]:
solution_data['Category'].value_counts()

In [ ]:
solution_data['Version'] = solution_data['Version'].astype(int)


In [ ]:
#solution_data_grouped - []
#solution_data_grouped = solution_data.groupby('Number').max('Version')

idx = solution_data.groupby(['Number'])['Version'].transform(max) == solution_data['Version']

solution_data_grouped = solution_data[idx]

In [ ]:
df_solution_data_grouped = pd.DataFrame(solution_data_grouped)

In [ ]:
def preprocess_txt(txt):
    txt = txt.lower()
    #punctuations = ''','''
    punctuations = '''!()[]{};:'"\,<>./?@#$%^&*_~'''
    for x in txt:
        if x in punctuations:
            txt = txt.replace(x, " ")
    txt = txt.split(" ")
    txt = ' '.join(x for x in txt if len(x)>=3)
    #txt = re.sub(r'\d',"#", txt)
    return(txt)
        

In [ ]:
solutionKeyTermsText = []
for d in df_solution_data_grouped['KeyTerms'][1001:1020]:
    #print(d)
    content = preprocess_txt(d)
    print(content)
    solutionKeyTermsText.append(content)


In [ ]:
%pip install sklearn --user
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
solutions_vectorizer = TfidfVectorizer(solutionKeyTermsText)
solutiona_KeyTerms_tfidf = solutions_vectorizer.fit_transform(solutionKeyTermsText)

In [ ]:
def get_tf_idf_query_similarity(vectorizer, docs_tfidf, query):
    """
    vectorizer: TfIdfVectorizer model
    docs_tfidf: tfidf vectors for all docs
    query: query doc

    return: cosine similarity between query and all docs
    """
    query_tfidf = vectorizer.transform([query])
    cosineSimilarities = cosine_similarity(query_tfidf, docs_tfidf).flatten()
    return cosineSimilarities

In [ ]:
get_tf_idf_query_similarity(solutions_vectorizer, solutiona_KeyTerms_tfidf, 'program  sapmgplv transactions  kp96  kp97')

# Testing the ISM (Intelligent Solution Matching) API

In [14]:
import requests
payload = {
    "queryId": "002028376500002067582021",
    "userId": "D123456",
    "systemId": "A11",
    "sessionId": "961779bd-726b-b62c-d035-73906fbf35db",
    "customerNumber": "S000000001",
    "customerName": "SAP AG",
    "installationNumber": "0020982966",
    "leadingProduct": " SAP NetWeaver  > Application Server ABAP ",
    "swType": "ONPREM",
    "swClass": "",
    "title": "SAP NetWeaver Application Server for ABAP",
    "problemDescription": "SAP NetWeaver Application Server for ABAP",
    "stepsToReproduce": "do this, then that",
    "refinementText": "more information here",
    "component": "BW-BEX-ET",
    "firstEntry": 0,
    "lastEntry": 2,
    "systemRole": "123",
    "otherCommunication": [
    {
      "date": "20210304T123001",
      "type": "refToSAP",
      "text": "setup wizard"
    },
    {
      "date": "20210304T123105",
      "type": "internalMemo",
      "text": "sample memo"
    }
    ],
    "supportPackages": [
    "SAPK-60853INSAPHRCSI",
    "SAPK-60853INSAPHRCSK",
    "SAPK-60853INSAPHRCTR"
    ],
    "productVersion": "SAP NetWeaver 7.2",
    "FilterSolutionType": [
    "note",
    "kba"
    ],
    "FilterSolutionStatus": [
    "Released for Customer",
    "Released Internally"
    ],
    "suggestKeywords": False
}
res = requests.post('http://ism-api-service-dev.ism-api.svc.cluster.local/',json=payload)
res_json = res.json()
res_json

ConnectionError: HTTPConnectionPool(host='ism-api-service-dev.ism-api.svc.cluster.local', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fdf69bed100>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))